In [15]:
# 9.1)

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

np.random.seed(0)

train = pd.read_csv("../input/titanic/train.csv")
test = pd.read_csv("../input/titanic/test.csv")

treino = train.copy()
teste = test.copy()

treino = treino.drop(['Cabin','Ticket','Name','PassengerId'],axis=1)
medianaIdade = treino['Age'].median()
medianaTarifa = treino['Fare'].median()
modaEmbarco = treino['Embarked'].mode()[0]
treino['Age'] = treino['Age'].fillna(medianaIdade)
treino['Fare'] = treino['Fare'].fillna(medianaTarifa)
treino['Embarked'] = treino['Embarked'].fillna(modaEmbarco)

teste = teste.drop(['Cabin','Ticket','Name','PassengerId'],axis=1)
teste['Age'] = teste['Age'].fillna(medianaIdade)
teste['Fare'] = teste['Fare'].fillna(medianaTarifa)
teste['Embarked'] = teste['Embarked'].fillna(modaEmbarco)

treino['AgeSpaced'],retbinsAge = pd.cut(treino['Age'],5,retbins=True)
retbinsAge[0] = 0
treino['FareBalanced'],retbinsFare = pd.qcut(treino['Fare'],6,retbins=True)
teste['AgeSpaced'] = pd.cut(teste['Age'],retbinsAge)
teste['FareBalanced'] = pd.cut(teste['Fare'],retbinsFare,include_lowest=True)

def onehot_encode(treino,teste,colunas):
    treinoAper = treino.copy()
    testeAper = teste.copy()
    
    enc = OneHotEncoder(handle_unknown = 'ignore',sparse = False)
    treinoCoded = pd.DataFrame(enc.fit_transform(treinoAper[colunas]))
    testeCoded = pd.DataFrame(enc.transform(testeAper[colunas]))
    
    treinoCoded.columns = enc.get_feature_names_out(colunas)
    testeCoded.columns = enc.get_feature_names_out(colunas)
    
    treinoAper = treinoAper.select_dtypes(exclude=['object'])
    testeAper = testeAper.select_dtypes(exclude=['object'])
    
    treinoAper = pd.concat([treinoAper, treinoCoded], axis=1)
    testeAper = pd.concat([testeAper, testeCoded], axis=1)
    return treinoAper, testeAper

def label_encode(treino,teste,colunas):
    treinoAper = treino.copy()
    testeAper = teste.copy()
    
    enc = LabelEncoder()
    treinoAper[colunas] = treino[colunas].apply(enc.fit_transform)
    testeAper[colunas] = teste[colunas].apply(enc.fit_transform)
    return treinoAper, testeAper

def miGraf(X,y,dis):
    mi = mutual_info_regression(X,y,random_state=0)
    mi = pd.DataFrame(mi, columns = ['Mutual Info'], index = dis)
    mi = mi.sort_values(by='Mutual Info',ascending = False)
    larg = np.arange(len(mi['Mutual Info']))
    plt.figure(dpi=200)
    plt.bar(larg, mi['Mutual Info'])
    plt.xticks(larg, list(mi.index),rotation=90)
    plt.title('Relação entre features e a saída')
    return mi

treino['AgeSpaced'],retbinsAge = pd.cut(treino['Age'],5,retbins=True)
retbinsAge[0] = 0
treino['FareBalanced'],retbinsFare = pd.qcut(treino['Fare'],6,retbins=True)

teste['AgeSpaced'] = pd.cut(teste['Age'],retbinsAge)
teste['FareBalanced'] = pd.cut(teste['Fare'],retbinsFare,include_lowest=True)

treinoAper = treino.drop(['Age','Fare'],axis=1)
testeAper = teste.drop(['Age','Fare'],axis=1)

colunas = list(testeAper.select_dtypes(include=['object']))
treinoAper, testeAper = onehot_encode(treinoAper,testeAper,colunas)

colunas = list(testeAper.select_dtypes(include=['category']))
treinoAper, testeAper = label_encode(treinoAper,testeAper,colunas)

X = treinoAper.drop(['Survived'],axis=1).astype('int64')
y = treinoAper['Survived']

Xtrain,Xval,ytrain,yval = train_test_split(X,y,test_size=0.20,random_state=0)
Xtest = testeAper.astype('int64')

dis = list(Xtrain.select_dtypes(include=['number']))
miGraf(X.values,y,dis)

In [16]:
import seaborn as sns

fig, (ax1, ax2) = plt.subplots(2,1,dpi=200)
plt.subplots_adjust(hspace=0.75)

sns.histplot(treino["Fare"], ax = ax1)
ax1.set_title('Distribuição total das taxas')
ax1.set_ylabel('Contagem')
ax1.set_xlabel('Tarifa')

sns.histplot(data = treino, x = "Fare", hue = "Survived", ax = ax2)
ax2.set_title('Distribuição relacionada com o target')
ax2.set_ylabel('Contagem')
ax2.set_xlabel('Tarifa')

print('A curva não é normal pois a distribuição não é simétrica.')
print('Nota-se que quanto maior a tarifa, maior a chance de ser um sobrevivente,')
print('visto que a cor laranja fica maior que a azul após cerca de 50.')

In [17]:
fig, ((ax1, ax2),(ax3,ax4)) = plt.subplots(2,2,dpi=200)
plt.subplots_adjust(left=-0.25,
                    bottom=0.1, 
                    right=0.65, 
                    top=0.9, 
                    wspace=0.75, 
                    hspace=0.75)

sns.histplot(data = treino, x = "Age", hue = "Sex", ax = ax1)
ax1.set_title('Distribuição da idade dos passageiros\nutilizando o sexo como camada')
ax1.set_ylabel('Contagem')
ax1.set_xlabel('Idade')

sns.histplot(data = treino.loc[treino['Survived']==1],
             x = "Age", hue = "Sex", ax = ax2)
ax2.set_title('A distribuição da idade dos passageiros,\napenas entre os sobreviventes,\nutilizando o sexo como camada')
ax2.set_ylabel('Contagem')
ax2.set_xlabel('Idade')

sns.histplot(data = treino.loc[treino['Sex']=='male'],
             x = "Age", hue = "Survived", ax = ax3)
ax3.set_title('Distribuição da idade dos homens,\nutilizando “Survived” como camada')
ax3.set_ylabel('Contagem')
ax3.set_xlabel('Idade')

sns.histplot(data = treino.loc[treino['Sex']=='female'],
             x = "Age", hue = "Survived", ax = ax4)
ax4.set_title('Distribuição da idade dos homens,\nutilizando “Survived” como camada')
ax4.set_ylabel('Contagem')
ax4.set_xlabel('Idade')

print('Mesmo havendo mais homens que mulheres, mais mulheres sobreviveram.')
print('O que resultou em um número maior de homens não sobreviventes e mulheres sobreviventes.')

In [18]:
# 9.2)

treino['Title'] = train['Name'].str.split(', ', expand = True)[1]\
.str.split('.', expand = True)[0]

teste['Title'] = test['Name'].str.split(', ', expand = True)[1]\
.str.split('.', expand = True)[0]

treino['TamFam'] = treino['SibSp']+treino['Parch']
teste['TamFam'] = teste['SibSp']+teste['Parch']

treino['Só'] = np.where(treino['TamFam'] == 0, True, False)
teste['Só'] = np.where(teste['TamFam'] == 0, True, False)

teste.head()

In [19]:
# 9.3)

from sklearn.cluster import KMeans

plt.figure(dpi=200)
plt.scatter(treino['Age'],treino['Fare'])
plt.xlabel('Idade')
plt.ylabel('Tarifa')

treinoK = treino[treino.Fare < 500].loc[:,['Age','Fare']]

kAF = KMeans(n_clusters=6)
treinoK['AgeFare'] = kAF.fit_predict(treinoK).astype('int64')

treino['AgeFare'] = treinoK['AgeFare']

In [20]:
treino.head()

In [21]:
plt.figure(dpi=200)
sns.scatterplot(data = treino, x = 'Fare', y = 'Age', hue = 'AgeFare')
plt.title('Distribuição “Fare” x “Age” tendo os grupos como camadas')
plt.ylabel('Age')
plt.xlabel('Fare')

plt.figure(dpi=200)
sns.barplot(data = treino, x = 'AgeFare', y = 'Survived')
plt.title('Distribuição “Fare” x “Age” tendo os grupos como camadas')
plt.ylabel('Contagem')
plt.xlabel('Age x Fare')

print('O grupo com maior sobrevivência foi o grupo com a 2ª maior tarifa,')
print('Sendo esse grupo presente em diversas idades.')
print('Já o grupo com menor sobrevivência foi o da faixa de tarifa mais barata')
print('Na idade jovem e adulta (10 anos a 40 anos).')

In [22]:
print(treino.loc[treino['AgeFare'].isnull()])
print('\nComo todos têm a tarifa cara, serão designados ao grupo 5.\n')
treino['AgeFare'] = treino['AgeFare'].fillna(5)
print(treino.loc[treino['Fare']>500])

In [23]:
treino['AgeSpaced'],retbinsAge = pd.cut(treino['Age'],5,retbins=True)
retbinsAge[0] = 0
treino['FareBalanced'],retbinsFare = pd.qcut(treino['Fare'],6,retbins=True)

teste['AgeSpaced'] = pd.cut(teste['Age'],retbinsAge)
teste['FareBalanced'] = pd.cut(teste['Fare'],retbinsFare,include_lowest=True)

treinoAper = treino.drop(['Age','Fare'],axis=1)
testeAper = teste.drop(['Age','Fare'],axis=1)

colunas = list(testeAper.select_dtypes(include=['object']))
treinoAper, testeAper = onehot_encode(treinoAper,testeAper,colunas)

colunas = list(testeAper.select_dtypes(include=['category']))
treinoAper, testeAper = label_encode(treinoAper,testeAper,colunas)

X = treinoAper.drop(['Survived'],axis=1).astype('int64')
y = treinoAper['Survived']

Xtrain,Xval,ytrain,yval = train_test_split(X,y,test_size=0.20,random_state=0)
Xtest = testeAper.astype('int64')

dis = list(Xtrain.select_dtypes(include=['number']))
miGraf(X.values,y,dis)

In [24]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

minmax = MinMaxScaler(copy = False)
features = ['Age','Fare']
Xsca = treino[features].copy()
for f in features:
    Xsca[[f]] = minmax.fit_transform(Xsca[[f]])

plt.figure(dpi=200)
sns.histplot(data = Xsca, x = 'Age')
plt.title('Histograma Idade')
plt.ylabel('Contagem')
plt.xlabel('Idade')

plt.figure(dpi=200)
sns.histplot(data = Xsca, x = 'Fare')
plt.title('Histograma Tarifa')
plt.ylabel('Contagem')
plt.xlabel('Tarifa')

In [25]:
pca = PCA()
Xpca = pd.DataFrame(pca.fit_transform(Xsca))

y = treino["Survived"]

dis = list(Xpca.select_dtypes(include=['number']))
miGraf(Xpca.values,y,dis)

In [28]:
def plot_variance(pca, width=8, dpi=100):
    # Create figure
    fig, axs = plt.subplots(1, 2)
    n = pca.n_components_
    grid = np.arange(1, n + 1)
    # Explained variance
    evr = pca.explained_variance_ratio_
    axs[0].bar(grid, evr)
    axs[0].set(
        xlabel="Component", title="% Explained Variance", ylim=(0.0, 1.0)
    )
    # Cumulative Variance
    cv = np.cumsum(evr)
    axs[1].plot(np.r_[0, grid], np.r_[0, cv], "o-")
    axs[1].set(
        xlabel="Component", title="% Cumulative Variance", ylim=(0.0, 1.0)
    )
    # Set up figure
    fig.set(figwidth=8, dpi=100)
    return axs

plot_variance(pca)

print('Não porque ambos os valores possuem variações relevantes para a saída.')

In [63]:
# 9.4)

from category_encoders import MEstimateEncoder

treinoAper['Title'] = treino['Title']

X = treinoAper.drop(['Survived','Só'],axis=1)
y = treinoAper['Survived']

Xtrain,Xenc,ytrain,yenc = train_test_split(X,y,test_size=0.20,random_state=0)

enc = MEstimateEncoder(cols=["Title"], m=5)
enc.fit(Xenc, yenc)
Xtrain = enc.transform(Xtrain)

plt.figure(dpi=200)
ax = sns.histplot(y, kde = False, stat="density")
ax = sns.kdeplot(Xtrain['Title'], color = 'r', ax = ax)
ax.set_label('Survived')
ax.legend(labels = ['Title', 'Survived'])

dis = list(Xtrain.select_dtypes(include=['number']))
miGraf(Xtrain.astype('int64').values,ytrain,dis)